In [1]:
using Distributed
# addprocs(4)

@everywhere begin
    using Revise
    using MendelIHT
    using SnpArrays
    using Random
    using GLM
    using DelimitedFiles
    using Test
    using Distributions
    using LinearAlgebra
    using CSV
    using DataFrames
end

┌ Info: Precompiling MendelIHT [921c7187-1484-5754-b919-5d3ed9ac03c4]
└ @ Base loading.jl:1278


# First simulate multivariate Gaussian traits

In [2]:
n = 1000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs
r = 2     # number of traits

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = simulate_random_snparray("multivariate_$(r)traits.bed", n, p)
xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true) 

# intercept is the only nongenetic covariate
z = ones(n, 1)
intercepts = [10.0 1.0] # each trait have different intercept

# simulate response y, true model b, and the correct non-0 positions of b
Y, true_Σ, true_b, correct_position = simulate_random_response(xla, k, r, Zu=z*intercepts, overlap=2);

In [3]:
# save true SNP's position and effect size
open("multivariate_$(r)traits_true_beta.txt", "w") do io
    println(io, "snpID,effectsize")
    for pos in correct_position
        println(io, "snp$pos,", true_b[pos])
    end
end

# create `.bim` and `.bam` files using phenotype
make_bim_fam_files(x, Y, "multivariate_$(r)traits")

# Run IHT

In [4]:
Yt = Matrix(Y')
Zt = Matrix(z')
ktrue = k + count(!iszero, intercepts)
@time result = fit_iht(Yt, Transpose(xla), Zt, k=11, verbose=true)

****                   MendelIHT Version 1.3.3                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) = 11
Prior weight scaling = off
Doubly sparse projection = off
Debias = off
Max IHT iterations = 100
Converging when tol < 0.0001:

Iteration 1: loglikelihood = 215.4892687838203, backtracks = 0, tol = 0.1258803451727764
Iteration 2: loglikelihood = 1400.4415131326584, backtracks = 0, tol = 0.027353391443577676
Iteration 3: loglikelihood = 1485.32487407949, backtracks = 0, tol = 0.007379616806130262
Iteration 4: loglikelihood = 1498.0290279587853, backtracks = 0, tol = 0.01194219


Compute time (sec):     1.1664929389953613
Final loglikelihood:    1514.818382728515
Iterations:             12

Trait 1: IHT estimated 3 nonzero SNP predictors
3×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │     5651    -0.200754
   2 │     5797    -1.09966
   3 │     8087     1.27901

Trait 1: IHT estimated 1 non-genetic predictors
1×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │        1       10.027

Trait 2: IHT estimated 6 nonzero SNP predictors
6×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │      326     0.331488
   2 │     2110     0.574476
   3 │     5375     1.19453
   4 │     5797     0.501675
   5 │     6015     0.81899
   6 │     6813     1.36762

Trait 2: IHT estimated 1 non-genetic predictors
1×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼──────────────

## Check answer

In [5]:
# first beta
β1 = result.beta[1, :]
true_b1_idx = findall(!iszero, true_b[:, 1])
[β1[true_b1_idx] true_b[true_b1_idx, 1]]

4×2 Array{Float64,2}:
 -0.200754  -0.224675
 -1.09966   -1.14044
  0.0       -0.14698
  1.27901    1.25668

In [6]:
# second beta
β2 = result.beta[2, :]
true_b2_idx = findall(!iszero, true_b[:, 2])
[β2[true_b2_idx] true_b[true_b2_idx, 2]]

6×2 Array{Float64,2}:
 0.331488  0.315219
 0.574476  0.609812
 1.19453   1.20121
 0.501675  0.531549
 0.81899   0.808327
 1.36762   1.43455

In [7]:
# non genetic covariates
[result.c intercepts']

2×2 Array{Float64,2}:
 10.027    10.0
  1.03625   1.0

In [8]:
# covariance matrix
[vec(result.Σ) vec(true_Σ)]

4×2 Array{Float64,2}:
  2.48446   2.53934
 -1.83826  -1.85399
 -1.83826  -1.85399
  2.42229   2.41416

In [13]:
# number of causal SNPs recovered
correct_snps = [x[1] for x in correct_position]  # truely causal snps
signif_snps = true_b1_idx ∪ true_b2_idx          # IHT's selected snps
signif_snps ∩ correct_snps

8-element Array{Int64,1}:
 5651
 5797
 6813
 8087
  326
 2110
 5375
 6015

# Test Cross validation

In [14]:
Random.seed!(2020)
Yt = Matrix(Y')
Zt = Matrix(z')
@time mses = cv_iht(Yt, x, Zt);



Crossvalidation Results:
	k	MSE
	1	2888.7160633632484
	2	2563.132216599166
	3	2082.0790076046433
	4	1823.5135405560698
	5	1569.64212209597
	6	1290.814635090646
	7	1168.4492977476905
	8	1112.8787725968111
	9	1030.5676713161718
	10	1033.800473194978
	11	1011.5326080855069
	12	1017.9517382638644
	13	1025.2244799284636
	14	1025.3849393164196
	15	1046.6759600584428
	16	1050.002316385212
	17	1033.9998469176733
	18	1032.590946556068
	19	1045.3508109946156
	20	1043.4650000999131
178.475397 seconds (10.48 M allocations: 917.849 MiB, 0.14% gc time)


In [15]:
argmin(mses)

11

**Conclusion:** After cross validation, IHT finds 10/12 predictors and 8/10 causal SNPs. 

# GEMMA multivariate results

In [9]:
gemma_df = CSV.read("gemma.result.assoc.txt", DataFrame)

# pvalues
pval_wald = gemma_df[!, :p_wald]
pval_lrt = gemma_df[!, :p_lrt]
pval_score = gemma_df[!, :p_score]

# estimated beta
estim_β1 = gemma_df[!, :beta_1]
estim_β2 = gemma_df[!, :beta_2]

# estimated covariance matrix
estim_σ11 = gemma_df[!, :Vbeta_1_1]
estim_σ12 = gemma_df[!, :Vbeta_1_2]
estim_σ22 = gemma_df[!, :Vbeta_2_2];

In [10]:
correct_snps = [x[1] for x in correct_position]  # truely causal snps
signif_snps = findall(x -> x ≤ 0.05/p, pval_lrt) # gemma's selected snps
signif_snps ∩ correct_snps

6-element Array{Int64,1}:
 2110
 5375
 5797
 6015
 6813
 8087

**Conclusion:** GEMMA finds 6/10 causal SNPs

# Compare with univariate IHT

+ `fit_iht`: ~20 times slower
+ `cv_iht`: ~70 times slower

In [49]:
n = 1000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs per trait
d = Normal
l = canonicallink(d())

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = simulate_random_snparray(undef, n, p)
xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true) 

# intercept is the only nongenetic covariate
z = ones(n)
intercept = 1.0

# simulate response y, true model b, and the correct non-0 positions of b
Y, true_b, correct_position = simulate_random_response(xla, k, d, l, Zu=z*intercept);

In [50]:
@time result = fit_iht(Y, xla, z, d=d(), l=l, k=11)

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse linear regression
Link functin = IdentityLink()
Sparsity parameter (k) = 11
Prior weight scaling = off
Doubly sparse projection = off
Debias = off
Debias = off
Max IHT iterations = 100
Converging when tol < 0.0001:

Iteration 1: loglikelihood = -1577.170794759688, backtracks = 0, tol = 0.609864675283163
Iteration 2: loglikelihood = -1484.8568136206177, backtracks = 0, tol = 0.1269955771967065
Iteration 3: loglikelihood = -1472.9529635904933, backtracks = 0, tol = 0.05823372413927707
Iteration 4: loglikelihood = -1472.5366421393842, backtracks = 1, tol = 0.00450895


IHT estimated 10 nonzero SNP predictors and 1 non-genetic predictors.

Compute time (sec):     0.06683206558227539
Final loglikelihood:    -1472.3905989669602
Iterations:             10

Selected genetic predictors:
10×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │      782    -0.437828
   2 │      901     0.747956
   3 │     1204     0.691327
   4 │     1306    -1.42505
   5 │     1655    -0.19456
   6 │     3160    -0.861591
   7 │     3936    -0.147235
   8 │     4201     0.338606
   9 │     4402    -0.126472
  10 │     6879    -1.21895

Selected nongenetic predictors:
1×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │        1      1.02016

In [41]:
# compare estimated vs true beta values
[result.beta[correct_position] true_b[correct_position]]

10×2 Array{Float64,2}:
 -0.437828  -0.402269
  0.747956   0.758756
  0.691327   0.729135
 -1.42505   -1.47163
 -0.19456   -0.172668
 -0.861591  -0.847906
  0.338606   0.296183
  0.0       -0.0034339
  0.0        0.125965
 -1.21895   -1.24972

### Cross validation timing

In [92]:
# 1 cores
Random.seed!(2020)
@time cv_iht(Y, x, z);



Crossvalidation Results:
	k	MSE
	1	1221.686764265696
	2	864.1553085156108
	3	661.6430376327284
	4	516.2679723147669
	5	418.2721191658746
	6	342.7931325462592
	7	303.15938053956916
	8	276.6241892508097
	9	278.29180795455045
	10	280.31878745464377
	11	282.10757732881564
	12	285.1294156734012
	13	293.16875008847467
	14	297.43739905389987
	15	296.17300065041906
	16	307.7814278347698
	17	302.0113818893564
	18	312.70262374274137
	19	309.87367962700966
	20	315.6750311115386
  4.315330 seconds (9.35 M allocations: 332.277 MiB, 1.12% gc time)
